## Model architecture definition

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class RecurrentCNN(nn.Module):
    def __init__(self, num_classes, lstm_hidden_size=256, dropout_prob=0.5):
        super(RecurrentCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((14, 56))
        self.lstm_input_size = 64 * 56
        self.lstm_hidden_size = lstm_hidden_size
        self.lstm = nn.LSTM(input_size=self.lstm_input_size, hidden_size=lstm_hidden_size,
                            batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(2 * lstm_hidden_size, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))      
        x = self.pool1(x)              
        x = F.relu(self.conv2(x))      
        x = self.pool2(x)              
        x = self.adaptive_pool(x)      
        x = x.permute(0, 2, 1, 3).contiguous()  
        batch_size, seq_len, channels, width = x.shape  
        x = x.view(batch_size, seq_len, channels * width)  
        lstm_out, _ = self.lstm(x)  
        x = lstm_out.mean(dim=1)    
        x = self.dropout(x)
        x = self.fc(x)
        return x

## Loading data

In [2]:
import cv2
import pandas as pd
import torch
import numpy as np

num_classes = 10

In [3]:
df_val = pd.read_csv('wikiart_csv/genre_val.csv',header=None, names=["image_path", "genre_id"])

In [4]:
val_images = df_val['image_path'].values
val_labels = df_val['genre_id'].values

In [5]:
# create test and train dataset for dataloader

def get_image(image_path,image_size=224):
    try:
        img = cv2.imread('./wikiart/' + image_path)
        if img is None:
            raise ValueError(f"Image not loaded: ./wikiart/{image_path}")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w, _ = img.shape
        scale = 256 / min(h, w)
        new_w = int(w * scale)
        new_h = int(h * scale)
        img_resized = cv2.resize(img, (new_w, new_h))
        start_x = (new_w - image_size) // 2
        start_y = (new_h - image_size) // 2
        img_cropped = img_resized[start_y:start_y+image_size, start_x:start_x+image_size]
        img_cropped = img_cropped.astype(np.float32) / 255.0
        img_tensor = torch.from_numpy(img_cropped).permute(2, 0, 1)
        mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        std  = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
        img_tensor = (img_tensor - mean) / std
        return img_tensor
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return torch.zeros(3, image_size, image_size)

class WikiArtDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # image_vectors = []
        # for image in self.images:
        #     image_emb = get_image(image)
        #     image_vectors.append(image_emb)
        # image = torch.stack(image_vectors)
        image = self.images[idx]
        # label should be a one-hot encoded vector
        label = torch.zeros(num_classes)
        label[self.labels[idx]] = 1

        return image, label

# train_dataset = WikiArtDataset(train_images, train_labels)
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataset = WikiArtDataset(val_images, val_labels)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=128, shuffle=False)

## Model declaration and loading saved weights

In [6]:
# Load the model
model = RecurrentCNN(num_classes=num_classes)
model.load_state_dict(torch.load('recurrent_cnn_epoch_5_genre.pth'))
model = model.cuda()

In [7]:
# Metrics to evaluate model performance

def top_1_accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    _, actual = torch.max(labels, 1)
    correct = (predicted == actual).sum().item()
    return correct / labels.shape[0]

def top_5_accuracy(outputs, labels):
    _, predicted = torch.topk(outputs, 5, dim=1)
    _, actual = torch.max(labels, 1)
    correct = 0
    for i in range(labels.shape[0]):
        if actual[i] in predicted[i]:
            correct += 1
    return correct / labels.shape[0]

# Evaluate the model on the validation set

model.eval()
top1_acc = 0
top5_acc = 0
num_batches = 0
for image_paths, labels in val_loader:
    with torch.no_grad():
        images = torch.stack([get_image(image_path) for image_path in image_paths])
        images = images.cuda()
        outputs = model(images)
        labels = labels.cuda()
        top1_acc += top_1_accuracy(outputs, labels)
        top5_acc += top_5_accuracy(outputs, labels)
        num_batches += 1

top1_acc /= num_batches
top5_acc /= num_batches
print(f"Top-1 accuracy: {top1_acc:.2f}")
print(f"Top-5 accuracy: {top5_acc:.2f}")


Top-1 accuracy: 0.56
Top-5 accuracy: 0.94


## Detecting outliers based on embeddings and confidence of prediction

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import cv2

softmax = nn.Softmax(dim=1)

def detect_outliers(outputs, labels, image_paths, threshold=0.5):
    probs = torch.nn.functional.softmax(outputs, dim=1)  
    max_probs, predicted_classes = torch.max(probs, dim=1)  
    _, actual_classes = torch.max(labels, dim=1)  

    outliers = []
    for i in range(len(image_paths)):
        prob = max_probs[i].item()  
        pred_class = predicted_classes[i].item()
        actual_class = actual_classes[i].item()

        if prob < threshold:
            outliers.append((image_paths[i], pred_class, actual_class, prob))

    return outliers


def get_embeddings(model, images):
    with torch.no_grad():
        x = model.conv1(images)
        x = F.relu(x)
        x = model.pool1(x)
        x = model.conv2(x)
        x = F.relu(x)
        x = model.pool2(x)
        x = model.adaptive_pool(x)
        x = x.permute(0, 2, 1, 3).contiguous()
        batch_size, seq_len, channels, width = x.shape
        x = x.view(batch_size, seq_len, channels * width)
        lstm_out, _ = model.lstm(x)
        return lstm_out.mean(dim=1).cpu().numpy()

# Load model
num_classes = 10
model = RecurrentCNN(num_classes=num_classes)
model.load_state_dict(torch.load('recurrent_cnn_epoch_5_genre.pth'))
model = model.cuda()
model.eval()

outlier_samples = []
all_embeddings = []
all_labels = []
all_image_paths = []
predictions = []

for image_paths, labels in val_loader:
    with torch.no_grad():
        images = torch.stack([get_image(image_path) for image_path in image_paths])
        images = images.cuda()
        outputs = model(images)
        labels = labels.cuda()
        
        # Detect outliers
        outliers = detect_outliers(outputs, labels, image_paths)
        outlier_samples.extend(outliers)
        
        t = torch.nn.functional.softmax(outputs, dim=1)
        max_probs, predicted_classes = torch.max(t, dim=1)
        predictions.extend(predicted_classes.cpu().numpy())
        # Collect embeddings for clustering
        embeddings = get_embeddings(model, images)
        all_embeddings.append(embeddings)
        all_labels.extend(labels.cpu().numpy())
        all_image_paths.extend(image_paths)

all_embeddings = np.vstack(all_embeddings)

similarity_matrix = cosine_similarity(all_embeddings)
outlier_indices = np.argsort(np.mean(similarity_matrix, axis=1))[:10]

print("Low-confidence outliers:")
outlier_accuracy = 0
for path, pred, actual, conf in outlier_samples:
    outlier_accuracy += (pred != actual)
    print(f"Image: {path}, Predicted: {pred}, Actual: {actual}, Confidence: {conf:.2f}")
    
outlier_accuracy /= len(outlier_samples)
print(f"Outlier detection accuracy by confidence: {outlier_accuracy:.2f}")

print("Embedding-based outliers:")
outlier_accuracy = 0
for idx in outlier_indices:
    outlier_accuracy += (predictions[idx] != all_labels[idx].argmax())
    print(f"Image: {all_image_paths[idx]}, Predicted: {predictions[idx]}, Actual: {all_labels[idx].argmax()}")

outlier_accuracy /= len(outlier_indices)
print(f"Outlier detection accuracy by embeddings: {outlier_accuracy:.2f}")


Low-confidence outliers:
Image: Impressionism/john-singer-sargent_at-torre-galli-ladies-in-a-garden-1910.jpg, Predicted: 4, Actual: 2, Confidence: 0.47
Image: Expressionism/pablo-picasso_big-heads-1969.jpg, Predicted: 6, Actual: 2, Confidence: 0.43
Image: Expressionism/egon-schiele_cardinal-and-nun-caress-1912.jpg, Predicted: 7, Actual: 2, Confidence: 0.47
Image: Impressionism/edgar-degas_the-curtain.jpg, Predicted: 6, Actual: 2, Confidence: 0.42
Image: Impressionism/constantin-artachino_on-the-terrace.jpg, Predicted: 2, Actual: 2, Confidence: 0.50
Image: Cubism/arthur-segal_die-melkerin.jpg, Predicted: 2, Actual: 2, Confidence: 0.49
Image: Naive_Art_Primitivism/marc-chagall_the-lovers-heaven-1964.jpg, Predicted: 0, Actual: 2, Confidence: 0.41
Image: Naive_Art_Primitivism/marc-chagall_dance-1962.jpg, Predicted: 6, Actual: 2, Confidence: 0.42
Image: Impressionism/john-singer-sargent_camping-at-lake-o-hara-1916.jpg, Predicted: 2, Actual: 2, Confidence: 0.50
Image: Impressionism/edgar-deg

## Statistical method to detect outliers based on mahalanobis distance

In [ ]:
import numpy as np

# Compute mean and covariance matrix for embeddings
mean_vec = np.mean(all_embeddings, axis=0)
cov_matrix = np.cov(all_embeddings, rowvar=False)
inv_cov_matrix = np.linalg.inv(cov_matrix)

def mahalanobis_distance(x, mean, inv_cov):
    delta = x - mean
    return np.sqrt(np.dot(np.dot(delta, inv_cov), delta.T))

# Calculate distances
distances = np.array([mahalanobis_distance(emb, mean_vec, inv_cov_matrix) for emb in all_embeddings])
# Set a threshold
threshold_distance = np.percentile(distances, 95)
mahalanobis_outliers = [all_image_paths[i] for i, d in enumerate(distances) if d > threshold_distance]

outlier_accuracy = 0
for path in mahalanobis_outliers:
    outlier_accuracy += 1 if predictions[all_image_paths.index(path)] != all_labels[all_image_paths.index(path)].argmax() else 0
    print(f"Outlier: {path} predicted as {predictions[all_image_paths.index(path)]}, actual {all_labels[all_image_paths.index(path)].argmax()}")

print(f"Outlier accuracy: {outlier_accuracy / len(mahalanobis_outliers):.2f}")


Outlier: Naive_Art_Primitivism/marc-chagall_dance-1962.jpg predicted as 6, actual 2
Outlier: Expressionism/dimitris-mytaras_untitled(1).jpg predicted as 6, actual 2
Outlier: Ukiyo_e/katsushika-hokusai_images-of-bathers-bathing-people.jpg predicted as 0, actual 2
Outlier: Post_Impressionism/paul-cezanne_harlequin-1890.jpg predicted as 6, actual 2
Outlier: Impressionism/theophrastos-triantafyllidis_bathers-on-the-boat-1930.jpg predicted as 7, actual 2
Outlier: Expressionism/edvard-munch_the-sick-child-ii-1896.jpg predicted as 6, actual 2
Outlier: Expressionism/henri-matisse_two-dancers-study-for-rouge-et-noir-1938.jpg predicted as 0, actual 2
Outlier: Impressionism/georges-seurat_stone-crusher.jpg predicted as 6, actual 2
Outlier: Art_Nouveau_Modern/raphael-kirchner_geisha-4.jpg predicted as 2, actual 2
Outlier: Cubism/jacques-villon_the-equilibrist-l-equilibriste-1913.jpg predicted as 9, actual 2
Outlier: Expressionism/lyonel-feininger_trumpeter-trompeter-1918.jpg predicted as 9, actual